# ファイル操作

In [1]:
import "os"
import "io/ioutil"
import "fmt"
var e error
var io *os.File

ファイル操作系のメソッドは基本的にエラーを発生させる可能性があり,各々のコマンドで `e` がエラーを受け取っている。  
エラーがあれば `e` にその内容が代入され,エラーがなければ `e==nil` になる。

* カレントディレクトリをホームに変更

In [2]:
home,e:=os.UserHomeDir()
os.Chdir(home)

ERROR: repl.go:1:9: not a type: os.UserHomeDir <*ast.SelectorExpr>

## ファイル/フォルダ/シンボリックリンクの作成

* Emptyという空フォルダを作成

In [3]:
e=os.Mkdir("Empty",0744)

* 一時ディレクトリの場合
```Go
var t string
t,e=ioutil.TempDir(os.TempDir,"uniqueIdentifier")
t // ディレクトリのパス
e
```

* Blankという空ファイルを作成

In [4]:
io,e=os.Create("Blank")
defer io.Close()

* フォルダEmptyの中にファイルBlankのシンボリックリンクSymlinkを作成

In [5]:
e=os.Symlink("../Blank","Empty/Symlink")

- ハードリンクの場合
```Go
e=os.Link("../Blank","Symlink")
```

## 書込み
* Untitled.mdというMarkdownファイルを作成して書込み

In [6]:
io,e=os.OpenFile("Untitled.md",os.O_WRONLY|os.O_CREATE,0644)
io.WriteString("# Header 1")
defer io.Close()
// or simply:
// ioutil.WriteFile("Untitled.md",[]byte("# Header 1"))

## 移動/名称変更

* フォルダEmptyをPackageに名称変更

In [7]:
e=os.Rename("Empty","Package")

* Packageフォルダ内のSymlinkファイルをAliasに名称変更

In [8]:
e=os.Rename("Package/Symlink","Package/Alias")

* Untitled.mdを移動して,名称変更

In [9]:
e=os.Rename("Untitled.md","Package/Headers.md")

## 追記
* Markdownファイルに追記

In [10]:
io,e=os.OpenFile("Package/Headers.md",os.O_WRONLY|os.O_APPEND,0644)
io.WriteString("\n## Header 2\n### Header 3")
defer io.Close()

## 読込み
* Markdownファイルを読込み

In [11]:
bytearray,e:=ioutil.ReadFile("Package/Headers.md")
string(bytearray)

# Header 1
## Header 2
### Header 3

## 再帰的にフォルダ作成
* フォルダを一気に作成

In [12]:
os.MkdirAll("Package/Module/Submodule/Item",0744)

- `os.Mkdir` の代わりに `os.MkdirAll` を使うことで,作成するフォルダItemの上位フォルダModule,Submoduleが存在していなくても,同時に生成される

## ディレクトリの内容を表示

In [13]:
io,e=os.Open("Package")
list,e:=io.Readdirnames(-1) // ファイルの個数を制限
e=io.Close()
fmt.Sprintf("%#v",list)

[]string{"Alias", "Module", "Headers.md"}

- `io.Readdir` なら内容が詳細に分かる `FileInfo` の配列を返す
- ワイルドカードを使って,条件を満たすファイルを見つけ出す場合は

```Go
import "path/filepath"
filepath.Glob("*.md")
```

## ファイル/フォルダの削除

* ファイルBlankを削除

In [14]:
os.Remove("Blank")

* シンボリックリンクAliasを削除

In [15]:
os.Remove("Package/Alias")

* フォルダModuleを削除

In [16]:
os.Remove("Package/Module")

remove Package/Module: directory not empty

- 空のフォルダは `os.Remove` で削除できる。空でない場合は削除できない。  
	`os.RemoveAll` を使えば,空でなくても削除できる

## 権限を確認/変更

* 状態を確認する関数

In [17]:
func check(path string) {
	info,e:=os.Lstat(path)
	if e!=nil {
		fmt.Println("存在していません")
		return
	}
	fmt.Println("存在しています")
	m:=info.Mode()
	if m&os.ModeSymlink != 0 {
		fmt.Println("シンボリックリンクです")
		link,e:=os.Readlink(path)
		if e!=nil { fmt.Println("リンク先:",link) }
		dstInfo,e:=os.Stat(path)
		if dstInfo.Mode().IsRegular() { fmt.Println("リンク先はファイルです") }
		if dstInfo.Mode().IsDir() { fmt.Println("リンク先はフォルダです") }
	} else
	{
		if m.IsRegular() { fmt.Println("ファイルです") }
		if m.IsDir() { fmt.Println("フォルダです") }
	}
	p:=m.Perm()
	if p&0400 != 0 { fmt.Println("読込可能です") }
	if p&0200 != 0 { fmt.Println("書込可能です") }
	if p&0100 != 0 { fmt.Println("実行可能です") }
	if info.Size()==0 { fmt.Println("空です") }
}

`os.Stat` でも情報が得られるが,ここでは `os.Lstat` を使う。なぜなら,シンボリックリンクに対して,前者はそのリンク先の情報を返すが,後者はそのリンク自体の情報を与えてくれるからだ。

* Markdownファイルの現在の状態を確認

In [18]:
check("Package/Headers.md")

存在しています
ファイルです
読込可能です
書込可能です


* 読込/書込権限を剥奪

In [19]:
os.Chmod("Package/Headers.md",0044)

* 状態を確認

In [20]:
check("Package/Headers.md")

存在しています
ファイルです


* 読込/実行権限を付加

In [21]:
os.Chmod("Package/Headers.md",0544)

* 状態を確認

In [22]:
check("Package/Headers.md")

存在しています
ファイルです
読込可能です
実行可能です


- `os.Chmod` は数値により権限を指定する
	* 4: 読込可能 (Readable)
	* 2: 書込可能 (Writable)
	* 1: 実行可能 (eXecutable)

- これらの和を3つ並べて指定する。  
    1つ目はユーザの権限,2つ目はゲストの権限,3つ目はその他の者の権限である。  
	Goでは8進数で表記するために,前に `0` を付加する

	e.g. `os.Chmod(somefile,0o754)`
	* ユーザは 7 = 4+2+1 だから,読込可能,書込可能,実行可能
	* ゲストは 5 = 4 + 1 だから,読込可能,実行可能
	* その他は 4 だから,読込のみ可能